# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [3]:
# Load API Credentials
with open('/Users/yawagipong/.secret/yelp_api.json', 'r') as f:
    login = json.load(f)

In [4]:
login.items()

dict_items([('client-id', 'DClgQxVVuDAybFmSzqfyKg'), ('api-key', 'Fewiokg7S9jhMiokwKeFI_dV8mII19sqqer4iwaQ5hYHc4MSzMWGTF1GeIKJrC42ByD5YYPqNCWmTYw9XnDq4mVo698xHnBkP17bmkyDdD6bqocsbt6Snvx-w-DXY3Yx')])

In [5]:
login.keys()

dict_keys(['client-id', 'api-key'])

In [6]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['api-key'], timeout_s = 5.0)

### Define Search Terms and File Paths

In [7]:
# set our API call parameters and filename before the first call
location = 'Seattle, WA 98122'
term = 'pizza'

In [8]:
## Specify fodler for saving data
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok = True)

In [9]:
#splits the location by commas, will just give the first part of dictionary
location.split(',')[0]

'Seattle'

In [10]:
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER + f"{location.split(',')[0]}-{term}.json"

In [11]:
JSON_FILE

'Data/Seattle-pizza.json'

### Check if Json File exists and Create it if it doesn't

In [12]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder) > 0:
        # create the folder
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
        json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists")


[i] Data/Seattle-pizza.json not found. Saving empty list to file.


### Load JSON FIle and account for previous results

In [13]:
## Load previous results and use len of results for offset
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)

#set offset based on previous results
n_results = len(previous_results)
print(f' - {n_results} previous results found.')

 - 0 previous results found.


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [14]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term = term, location = location)

In [15]:
type(results)

dict

In [16]:
## How many results total?
len(results)
#3 means there are 3 keys

3

- Where is the actual data we want to save?

In [17]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [18]:
results['businesses']

[{'id': 't-Z9bvmlgUyDtomGmttrUQ',
  'alias': 'guerilla-pizza-kitchen-seattle',
  'name': 'Guerilla Pizza Kitchen',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEjc9J_4Tb1PtAb71UA/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/guerilla-pizza-kitchen-seattle?adjust_creative=DClgQxVVuDAybFmSzqfyKg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=DClgQxVVuDAybFmSzqfyKg',
  'review_count': 1,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'},
   {'alias': 'popuprestaurants', 'title': 'Pop-Up Restaurants'}],
  'rating': 5.0,
  'coordinates': {'latitude': 47.608127, 'longitude': -122.302435},
  'transactions': [],
  'location': {'address1': '2300 E Cherry St',
   'address2': '',
   'address3': None,
   'city': 'Seattle',
   'zip_code': '98122',
   'country': 'US',
   'state': 'WA',
   'display_address': ['2300 E Cherry St', 'Seattle, WA 98122']},
  'phone': '',
  'display_phone': '',
  'distance': 749.1731129459927},
 {'id': 'EwrAoy7Xb

In [19]:
results['total']
#856 records exist for Seattle pizzas

856

In [20]:
pd.DataFrame(results['businesses'])

id                                          alias  \
0   t-Z9bvmlgUyDtomGmttrUQ                 guerilla-pizza-kitchen-seattle   
1   EwrAoy7XbX8nfemZdDvupA                               cornelly-seattle   
2   FVbwpNA1uZEGiM02N4XtUg                                 blotto-seattle   
3   fxyHWmfzcdjImgQ_UYHoTw                              bar-cotto-seattle   
4   IrohtoYjnAR_vc6w6CRCxA                                   moto-seattle   
5   Wi6LFkjIausYj277ru6pqg                             humble-pie-seattle   
6   M9xzvwgK58T0w7wvXedvuQ                        hot-mamas-pizza-seattle   
7   et4eWK49x5od4lfqCukG6w                          central-pizza-seattle   
8   cAFXng3Gr1cBCFkP1VIxng             the-independent-pizzeria-seattle-2   
9   Ss_66zV-zhK_57_6MgV93w                           aglio-e-olio-seattle   
10  wi-mbKEz4Ha1YYaOB8zHnw                             kōbo-pizza-seattle   
11  kVqwgAjEPffEyMJSzpfM_A                     pagliacci-pizza-seattle-26   
12  ugTsEtjvwRhteac_6JcuPw                   italian-family-pizza-seattle   
13  -FOAQv22SXtSBs7nptI3UA                 serious-pie-downtown-seattle-2   
14  GJdmwqqkC0nAQfX1qZM3ew  olympia-pizza-and-spaghetti-house-iii-seattle   
15  n6BLr6spgjVWqzUwmVx0bg                    johnny-mos-pizzeria-seattle   
16  k3riaoflafifFUSNKXbfTw             via-tribunali-capitol-hill-seattle   
17  1rkgyfwzXG7qSdVZdfRt7g                                 roccos-seattle   
18  PpAiLhErwd08QhPb06_-hA                             mioposto-seattle-7   
19  QeeiVw_OjxWmTiSH0A6qgA                             rione-xiii-seattle   

                                     name  \
0                  Guerilla Pizza Kitchen   
1                                Cornelly   
2                                  Blotto   
3                               Bar Cotto   
4                                    Moto   
5                              Humble Pie   
6                        Hot Mama's Pizza   
7                           Central Pizza   
8                The Independent Pizzeria   
9                            Aglio e Olio   
10                             Kōbo Pizza   
11                        Pagliacci Pizza   
12                   Italian Family Pizza   
13                   Serious Pie Downtown   
14  Olympia Pizza and Spaghetti House III   
15                   Johnny Mo's Pizzeria   
16           Via Tribunali - Capitol Hill   
17                                Rocco's   
18                               Mioposto   
19                             Rione XIII   

                                            image_url  is_closed  \
0   https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...      False   
1   https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...      False   
2   https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...      False   
3   https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...      False   
4   https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...      False   
5   https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...      False   
6   https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...      False   
7   https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...      False   
8   https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...      False   
9   https://s3-media2.fl.yelpcdn.com/bphoto/inrSl1...      False   
10  https://s3-media1.fl.yelpcdn.com/bphoto/6et4T9...      False   
11  https://s3-media3.fl.yelpcdn.com/bphoto/7thHN-...      False   
12  https://s3-media2.fl.yelpcdn.com/bphoto/SgiSrJ...      False   
13  https://s3-media2.fl.yelpcdn.com/bphoto/dy0pJ5...      False   
14  https://s3-media1.fl.yelpcdn.com/bphoto/KvTgMA...      False   
15  https://s3-media4.fl.yelpcdn.com/bphoto/8IAnDP...      False   
16  https://s3-media4.fl.yelpcdn.com/bphoto/8nqcG7...      False   
17  https://s3-media3.fl.yelpcdn.com/bphoto/AWBXTE...      False   
18  https://s3-media1.fl.yelpcdn.com/bphoto/gWBw_l...      False   
19  https://s3-media2.fl.yelpcdn.com/bphoto/ENHXwc...      False   

                               

In [21]:
## How many did we get the details for?
results_per_page =  len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [22]:
# Use math.ceil to round up for the total number of pages of results.
import time, math

n_pages = math.ceil((results['total'])/ results_per_page)
n_pages

43

In [29]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        time.sleep(.2)
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(location = location,
                                   term = term,
                                   offset = n_results + 1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])
        
        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    ## What to do if we get an error/exception.
    except Exception as e:
        print('[!] ERROR:', e)


  0%|          | 0/43 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [30]:
df = pd.read_json(JSON_FILE)
df.head()

id               alias        name  \
0  EwrAoy7XbX8nfemZdDvupA    cornelly-seattle    Cornelly   
1  FVbwpNA1uZEGiM02N4XtUg      blotto-seattle      Blotto   
2  fxyHWmfzcdjImgQ_UYHoTw   bar-cotto-seattle   Bar Cotto   
3  IrohtoYjnAR_vc6w6CRCxA        moto-seattle        Moto   
4  Wi6LFkjIausYj277ru6pqg  humble-pie-seattle  Humble Pie   

                                           image_url  is_closed  \
0  https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...      False   
1  https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...      False   
2  https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...      False   
3  https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...      False   
4  https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...      False   

                                                 url  review_count  \
0  https://www.yelp.com/biz/cornelly-seattle?adju...           101   
1  https://www.yelp.com/biz/blotto-seattle?adjust...            40   
2  https://www.yelp.com/biz/bar-cotto-seattle?adj...           343   
3  https://www.yelp.com/biz/moto-seattle?adjust_c...           143   
4  https://www.yelp.com/biz/humble-pie-seattle?ad...           362   

                                          categories  rating  \
0  [{'alias': 'pizza', 'title': 'Pizza'}, {'alias...     4.5   
1             [{'alias': 'pizza', 'title': 'Pizza'}]     4.5   
2  [{'alias': 'bars', 'title': 'Bars'}, {'alias':...     4.5   
3  [{'alias': 'pizza', 'title': 'Pizza'}, {'alias...     4.5   
4             [{'alias': 'pizza', 'title': 'Pizza'}]     4.0   

                                         coordinates        transactions  \
0  {'latitude': 47.624577, 'longitude': -122.325577}  [delivery, pickup]   
1    {'latitude': 47.61849, 'longitude': -122.31664}                  []   
2  {'latitude': 47.6146934, 'longitude': -122.312...  [delivery, pickup]   
3    {'latitude': 47.56207, 'longitude': -122.38509}          [delivery]   
4  {'latitude': 47.5976491915013, 'longitude': -1...          [delivery]   

                                            location         phone  \
0  {'address1': '601 Summit Ave E', 'address2': '...                 
1  {'address1': '1830 12th Ave', 'address2': '', ...  +12064031809   
2  {'address1': '1546 15th Ave', 'address2': '', ...  +12068388081   
3  {'address1': '4526 42nd Ave SW', 'address2': '...  +12064208880   
4  {'address1': '525 Rainier Ave S', 'address2': ...  +12063295133   

    display_phone     distance price  
0                  2578.701393   NaN  
1  (206) 403-1809  1686.862176   NaN  
2  (206) 838-8081  1292.526796    $$  
3  (206) 420-8880  8784.075722    $$  
4  (206) 329-5133  2166.834011    $$

In [31]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Seattle-pizza.csv.gz'

In [32]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression = 'gzip', index = False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [33]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 834,859 Bytes
CSV.GZ FILE: 120,943 Bytes
the csv.gz is 6.902912942460498 times smaller!


## Next Class: Processing the Results and Mapping 